In [1]:
import pandas as pd
import polars as pl
import duckdb as db

In [2]:
import numpy as np

In [3]:
# Pandas

In [4]:
df = pd.read_parquet('../AwesomeDAG/dags/data/names.parquet').reset_index()

In [5]:
df['birthYear'].isna().mean()

0.9536769252345741

In [6]:
df.loc[df['birthYear'].isna(),'birthYear'] = np.random.randint(df['birthYear'].quantile(.25),df['birthYear'].max(),df['birthYear'].isna().sum())
df.loc[df['deathYear'].isna(),'deathYear'] = df['birthYear'] + np.random.randint(12,80,df.shape[0])

In [7]:
%%timeit
df.groupby(['birthYear',df['deathYear'].isna(),'primaryProfession_actor']).agg({'nconst':'count','primaryProfession_director':'mean'})

468 ms ± 26.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# Polars

In [9]:
df_pl = pl.read_parquet('../AwesomeDAG/dags/data/names.parquet')

In [10]:
df_pl = pl.from_dataframe(df.drop('knownForTitles',axis=1))

In [11]:
%%timeit
df_pl.groupby(['birthYear',pl.col('deathYear').is_null(),'primaryProfession_actor']).agg([pl.col('nconst').count(),pl.col('primaryProfession_director').mean()])

349 ms ± 25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# duckdb

In [13]:
conn = db.connect()
df_db = conn.execute('''
                    create temp table data
                    as
                    select 
                    *
                    from read_parquet('../AwesomeDAG/dags/data/names.parquet')
                    ''')
conn.close()

In [14]:
conn = db.connect()
df_db = conn.execute('''
                    create temp table my_data as select * from df
                    ''')

In [17]:
%%timeit
df_db.query('''

            select 
            birthYear,
            deathYear is NULL as is_alive,
            primaryProfession_actor,
            count(nconst) as cnt,
            avg(primaryProfession_director) as avg_dir_act
            from my_data
            group by 1,2,3

            ''')

70.8 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
%%timeit
df_db.query('''

            select 
            birthYear,
            deathYear is NULL as is_alive,
            primaryProfession_actor,
            count(nconst) as cnt,
            avg(primaryProfession_director) as avg_dir_act
            from my_data
            group by 1,2,3

            ''').df()

39.8 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
